<a href="https://colab.research.google.com/github/carolhcs/ML-Engineering-Data-Science/blob/main/PowerBI_Python_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Desafio de Projeto criado durante o bootcamp de Python Data Analytics da DIO
### por Hellen Caroline Salvato
💻
Os desafios originais foram feitos usando o Power BI da Microsoft, que é bem mais imples e amigavel. Porém como existem varias limitações relacionadas a ele, resolvi refazer meu desafio em python, como uma forma de me desafiar.


Vamos utilizar as seguintes ferramentas:

🐼
**Pandas** para manipulação de dados.

📊
**Plotly** para visualizações interativas.

💨
**Dash** para criar o dashboard interativo.

### 1. Criando Elementos da Primeira Página do Relatório
**Carregando e Preparando os Dados**

Primeiro, vamos carregar os dados de vendas e preparar a estrutura do nosso relatório.

In [ ]:
import pandas as pd

# Carregar os dados de vendas
df = pd.read_csv('sales_data.csv')  # Substitua pelo seu arquivo de dados

# Exemplo de visualização inicial dos dados
print(df.head())


### 2. Criando Gráficos da Primeira Página do Relatório
**Gráfico de Vendas por Categoria**

In [ ]:
import plotly.express as px

fig = px.bar(df, x='Category', y='Sales', title='Vendas por Categoria')
fig.show()


### 3. Criando o Segmentador e Visuais Alternativos
**Segmentador por Ano**

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1('Relatório Gerencial de Vendas'),
    dcc.Dropdown(
        id='year-dropdown',
        options=[{'label': year, 'value': year} for year in df['Year'].unique()],
        value=df['Year'].unique()[0]
    ),
    dcc.Graph(id='sales-by-category')
])

@app.callback(
    Output('sales-by-category', 'figure'),
    [Input('year-dropdown', 'value')]
)
def update_graph(selected_year):
    filtered_df = df[df['Year'] == selected_year]
    fig = px.bar(filtered_df, x='Category', y='Sales', title=f'Vendas por Categoria em {selected_year}')
    return fig


### 4. Criando os Botões do Relatório e Utilizando Indicadores para Gravar Estado do Relatório
**Botões para Alternar Entre Gráficos**

In [ ]:
app.layout = html.Div([
    html.H1('Relatório Gerencial de Vendas'),
    dcc.Dropdown(
        id='year-dropdown',
        options=[{'label': year, 'value': year} for year in df['Year'].unique()],
        value=df['Year'].unique()[0]
    ),
    dcc.Tabs(id='tabs', value='tab-1', children=[
        dcc.Tab(label='Vendas por Categoria', value='tab-1'),
        dcc.Tab(label='Vendas por Região', value='tab-2')
    ]),
    html.Div(id='tabs-content')
])

@app.callback(
    Output('tabs-content', 'children'),
    [Input('tabs', 'value'), Input('year-dropdown', 'value')]
)
def render_content(tab, selected_year):
    filtered_df = df[df['Year'] == selected_year]
    if tab == 'tab-1':
        fig = px.bar(filtered_df, x='Category', y='Sales', title=f'Vendas por Categoria em {selected_year}')
    elif tab == 'tab-2':
        fig = px.bar(filtered_df, x='Region', y='Sales', title=f'Vendas por Região em {selected_year}')
    return dcc.Graph(figure=fig)


### 5. Criando a Interação por Botões com a Segunda Página do Relatório
**Navegação entre Páginas**

In [ ]:
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])

index_page = html.Div([
    html.H1('Relatório Gerencial de Vendas'),
    dcc.Link('Ir para Primeira Página', href='/page-1'),
    html.Br(),
    dcc.Link('Ir para Segunda Página', href='/page-2')
])

page_1_layout = html.Div([
    html.H1('Primeira Página'),
    dcc.Dropdown(
        id='year-dropdown',
        options=[{'label': year, 'value': year} for year in df['Year'].unique()],
        value=df['Year'].unique()[0]
    ),
    dcc.Graph(id='sales-by-category'),
    html.Br(),
    dcc.Link('Voltar para Home', href='/')
])

page_2_layout = html.Div([
    html.H1('Segunda Página'),
    dcc.Graph(id='sales-by-region'),
    html.Br(),
    dcc.Link('Voltar para Home', href='/')
])

@app.callback(
    Output('page-content', 'children'),
    [Input('url', 'pathname')]
)
def display_page(pathname):
    if pathname == '/page-1':
        return page_1_layout
    elif pathname == '/page-2':
        return page_2_layout
    else:
        return index_page

@app.callback(
    Output('sales-by-category', 'figure'),
    [Input('year-dropdown', 'value')]
)
def update_graph(selected_year):
    filtered_df = df[df['Year'] == selected_year]
    fig = px.bar(filtered_df, x='Category', y='Sales', title=f'Vendas por Categoria em {selected_year}')
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)


### 6. Publicando Relatório no Power BI Service

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False, host='0.0.0.0', port=80)  # Configuração para rodar em um servidor
